In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time
from contextlib import contextmanager

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier

import re

import warnings
warnings.filterwarnings('ignore')

In [7]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))


# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


def do_sum(dataframe, group_cols, counted, agg_name):
    gp = dataframe[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(columns={counted: agg_name})
    dataframe = dataframe.merge(gp, on=group_cols, how='left')
    return dataframe


def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()


def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)


# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

In [8]:
path = '../Données/'
nan_as_category = True

In [9]:
# Importation des jeux de données
train_df = pd.read_csv(path + 'application_train.csv')
print(train_df.shape)
test_df = pd.read_csv(path + 'application_test.csv')
print(test_df.shape)
df_bureau = pd.read_csv(path + 'bureau.csv')
print(df_bureau.shape)
df_bureau_balance = pd.read_csv(path + 'bureau_balance.csv')
print(df_bureau_balance.shape)
df_previous = pd.read_csv(path + 'previous_application.csv')
print(df_previous.shape)
df_pos_cash = pd.read_csv(path + 'POS_CASH_balance.csv')
print(df_pos_cash.shape)
df_installments = pd.read_csv(path + 'installments_payments.csv')
print(df_installments.shape)
df_credit_card = pd.read_csv(path + 'credit_card_balance.csv')
print(df_credit_card.shape)

(307511, 122)
(48744, 121)
(1716428, 17)
(27299925, 3)
(1670214, 37)
(10001358, 8)
(13605401, 8)
(3840312, 23)


In [10]:
def get_age_label(days_birth):
    """ Return the age group label (int). """
    age_years = -days_birth / 365
    if age_years < 27: return 1
    elif age_years < 40: return 2
    elif age_years < 50: return 3
    elif age_years < 65: return 4
    elif age_years < 99: return 5
    else: return 0

# Preprocess application_train.csv and application_test.csv
with timer("Process application_train"):
    df_applications = train_df
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df_applications = df_applications[df_applications['CODE_GENDER'] != 'XNA']
    df_applications = df_applications[df_applications['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df_applications['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    df_applications['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df_applications[bin_feature], uniques = pd.factorize(df_applications[bin_feature])
    # Categorical features with One-Hot encode
    df_applications, cat_cols = one_hot_encoder(df_applications, nan_as_category)

    # Flag_document features - count and kurtosis
    docs = [f for f in df_applications.columns if 'FLAG_DOC' in f]
    df_applications['DOCUMENT_COUNT'] = df_applications[docs].sum(axis=1)
    df_applications['NEW_DOC_KURT'] = df_applications[docs].kurtosis(axis=1)

    # Categorical age - based on target=1 plot
    df_applications['AGE_RANGE'] = df_applications['DAYS_BIRTH'].apply(lambda x: get_age_label(x))

    # New features based on External sources
    df_applications['EXT_SOURCES_PROD'] = df_applications['EXT_SOURCE_1'] * df_applications['EXT_SOURCE_2'] * df_applications['EXT_SOURCE_3']
    df_applications['EXT_SOURCES_WEIGHTED'] = df_applications.EXT_SOURCE_1 * 2 + df_applications.EXT_SOURCE_2 * 1 + df_applications.EXT_SOURCE_3 * 3
    np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
    for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        df_applications[feature_name] = eval('np.{}'.format(function_name))(
            df_applications[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

    # Some simple new features (percentages)
    df_applications['DAYS_EMPLOYED_PERC'] = df_applications['DAYS_EMPLOYED'] / df_applications['DAYS_BIRTH']
    df_applications['INCOME_CREDIT_PERC'] = df_applications['AMT_INCOME_TOTAL'] / df_applications['AMT_CREDIT']
    df_applications['INCOME_PER_PERSON'] = df_applications['AMT_INCOME_TOTAL'] / df_applications['CNT_FAM_MEMBERS']
    df_applications['ANNUITY_INCOME_PERC'] = df_applications['AMT_ANNUITY'] / df_applications['AMT_INCOME_TOTAL']
    df_applications['PAYMENT_RATE'] = df_applications['AMT_ANNUITY'] / df_applications['AMT_CREDIT']

    # Credit ratios
    df_applications['CREDIT_TO_GOODS_RATIO'] = df_applications['AMT_CREDIT'] / df_applications['AMT_GOODS_PRICE']

    # Income ratios
    df_applications['INCOME_TO_EMPLOYED_RATIO'] = df_applications['AMT_INCOME_TOTAL'] / df_applications['DAYS_EMPLOYED']
    df_applications['INCOME_TO_BIRTH_RATIO'] = df_applications['AMT_INCOME_TOTAL'] / df_applications['DAYS_BIRTH']

    # Time ratios
    df_applications['ID_TO_BIRTH_RATIO'] = df_applications['DAYS_ID_PUBLISH'] / df_applications['DAYS_BIRTH']
    df_applications['CAR_TO_BIRTH_RATIO'] = df_applications['OWN_CAR_AGE'] / df_applications['DAYS_BIRTH']
    df_applications['CAR_TO_EMPLOYED_RATIO'] = df_applications['OWN_CAR_AGE'] / df_applications['DAYS_EMPLOYED']
    df_applications['PHONE_TO_BIRTH_RATIO'] = df_applications['DAYS_LAST_PHONE_CHANGE'] / df_applications['DAYS_BIRTH']

    # EXT_SOURCE_X FEATURE
    df_applications['APPS_EXT_SOURCE_MEAN'] = df_applications[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df_applications['APPS_EXT_SOURCE_STD'] = df_applications[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df_applications['APPS_EXT_SOURCE_STD'] = df_applications['APPS_EXT_SOURCE_STD'].fillna(df_applications['APPS_EXT_SOURCE_STD'].mean())
    df_applications['APP_SCORE1_TO_BIRTH_RATIO'] = df_applications['EXT_SOURCE_1'] / (df_applications['DAYS_BIRTH'] / 365.25)
    df_applications['APP_SCORE2_TO_BIRTH_RATIO'] = df_applications['EXT_SOURCE_2'] / (df_applications['DAYS_BIRTH'] / 365.25)
    df_applications['APP_SCORE3_TO_BIRTH_RATIO'] = df_applications['EXT_SOURCE_3'] / (df_applications['DAYS_BIRTH'] / 365.25)
    df_applications['APP_SCORE1_TO_EMPLOY_RATIO'] = df_applications['EXT_SOURCE_1'] / (df_applications['DAYS_EMPLOYED'] / 365.25)
    df_applications['APP_EXT_SOURCE_2*EXT_SOURCE_3*DAYS_BIRTH'] = df_applications['EXT_SOURCE_1'] * df_applications['EXT_SOURCE_2'] * df_applications['DAYS_BIRTH']
    df_applications['APP_SCORE1_TO_FAM_CNT_RATIO'] = df_applications['EXT_SOURCE_1'] / df_applications['CNT_FAM_MEMBERS']
    df_applications['APP_SCORE1_TO_GOODS_RATIO'] = df_applications['EXT_SOURCE_1'] / df_applications['AMT_GOODS_PRICE']
    df_applications['APP_SCORE1_TO_CREDIT_RATIO'] = df_applications['EXT_SOURCE_1'] / df_applications['AMT_CREDIT']
    df_applications['APP_SCORE1_TO_SCORE2_RATIO'] = df_applications['EXT_SOURCE_1'] / df_applications['EXT_SOURCE_2']
    df_applications['APP_SCORE1_TO_SCORE3_RATIO'] = df_applications['EXT_SOURCE_1'] / df_applications['EXT_SOURCE_3']
    df_applications['APP_SCORE2_TO_CREDIT_RATIO'] = df_applications['EXT_SOURCE_2'] / df_applications['AMT_CREDIT']
    df_applications['APP_SCORE2_TO_REGION_RATING_RATIO'] = df_applications['EXT_SOURCE_2'] / df_applications['REGION_RATING_CLIENT']
    df_applications['APP_SCORE2_TO_CITY_RATING_RATIO'] = df_applications['EXT_SOURCE_2'] / df_applications['REGION_RATING_CLIENT_W_CITY']
    df_applications['APP_SCORE2_TO_POP_RATIO'] = df_applications['EXT_SOURCE_2'] / df_applications['REGION_POPULATION_RELATIVE']
    df_applications['APP_SCORE2_TO_PHONE_CHANGE_RATIO'] = df_applications['EXT_SOURCE_2'] / df_applications['DAYS_LAST_PHONE_CHANGE']
    df_applications['APP_EXT_SOURCE_1*EXT_SOURCE_2'] = df_applications['EXT_SOURCE_1'] * df_applications['EXT_SOURCE_2']
    df_applications['APP_EXT_SOURCE_1*EXT_SOURCE_3'] = df_applications['EXT_SOURCE_1'] * df_applications['EXT_SOURCE_3']
    df_applications['APP_EXT_SOURCE_2*EXT_SOURCE_3'] = df_applications['EXT_SOURCE_2'] * df_applications['EXT_SOURCE_3']
    df_applications['APP_EXT_SOURCE_1*DAYS_EMPLOYED'] = df_applications['EXT_SOURCE_1'] * df_applications['DAYS_EMPLOYED']
    df_applications['APP_EXT_SOURCE_2*DAYS_EMPLOYED'] = df_applications['EXT_SOURCE_2'] * df_applications['DAYS_EMPLOYED']
    df_applications['APP_EXT_SOURCE_3*DAYS_EMPLOYED'] = df_applications['EXT_SOURCE_3'] * df_applications['DAYS_EMPLOYED']

    # AMT_INCOME_TOTAL : income
    # CNT_FAM_MEMBERS  : the number of family members
    df_applications['APPS_GOODS_INCOME_RATIO'] = df_applications['AMT_GOODS_PRICE'] / df_applications['AMT_INCOME_TOTAL']
    df_applications['APPS_CNT_FAM_INCOME_RATIO'] = df_applications['AMT_INCOME_TOTAL'] / df_applications['CNT_FAM_MEMBERS']

    # DAYS_BIRTH : Client's age in days at the time of application
    # DAYS_EMPLOYED : How many days before the application the person started current employment
    df_applications['APPS_INCOME_EMPLOYED_RATIO'] = df_applications['AMT_INCOME_TOTAL'] / df_applications['DAYS_EMPLOYED']

    # other feature from better than 0.8
    df_applications['CREDIT_TO_GOODS_RATIO_2'] = df_applications['AMT_CREDIT'] / df_applications['AMT_GOODS_PRICE']
    df_applications['APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio'] = df_applications['AMT_INCOME_TOTAL'] / 12. - df_applications['AMT_ANNUITY']
    df_applications['APP_INCOME_TO_EMPLOYED_RATIO'] = df_applications['AMT_INCOME_TOTAL'] / df_applications['DAYS_EMPLOYED']
    df_applications['APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio'] = df_applications['DAYS_LAST_PHONE_CHANGE'] / df_applications['DAYS_EMPLOYED']
    df_applications['APP_DAYS_EMPLOYED_DAYS_BIRTH_diff'] = df_applications['DAYS_EMPLOYED'] - df_applications['DAYS_BIRTH']

    print('"Application_Train" final shape:', df_applications.shape)

"Application_Train" final shape: (307506, 308)
Process application_train - done in 3s


In [11]:
# Preprocess bureau.csv and bureau_balance.csv
with timer("Process bureau and bureau_balance"):
    # Credit duration and credit/account end date difference
    df_bureau['CREDIT_DURATION'] = -df_bureau['DAYS_CREDIT'] + df_bureau['DAYS_CREDIT_ENDDATE']
    df_bureau['ENDDATE_DIF'] = df_bureau['DAYS_CREDIT_ENDDATE'] - df_bureau['DAYS_ENDDATE_FACT']
    
    # Credit to debt ratio and difference
    df_bureau['DEBT_PERCENTAGE'] = df_bureau['AMT_CREDIT_SUM'] / df_bureau['AMT_CREDIT_SUM_DEBT']
    df_bureau['DEBT_CREDIT_DIFF'] = df_bureau['AMT_CREDIT_SUM'] - df_bureau['AMT_CREDIT_SUM_DEBT']
    df_bureau['CREDIT_TO_ANNUITY_RATIO'] = df_bureau['AMT_CREDIT_SUM'] / df_bureau['AMT_ANNUITY']
    df_bureau['BUREAU_CREDIT_FACT_DIFF'] = df_bureau['DAYS_CREDIT'] - df_bureau['DAYS_ENDDATE_FACT']
    df_bureau['BUREAU_CREDIT_ENDDATE_DIFF'] = df_bureau['DAYS_CREDIT'] - df_bureau['DAYS_CREDIT_ENDDATE']
    df_bureau['BUREAU_CREDIT_DEBT_RATIO'] = df_bureau['AMT_CREDIT_SUM_DEBT'] / df_bureau['AMT_CREDIT_SUM']

    # CREDIT_DAY_OVERDUE :
    df_bureau['BUREAU_IS_DPD'] = df_bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)
    df_bureau['BUREAU_IS_DPD_OVER120'] = df_bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 120 else 0)

    df_bureau_balance, bb_cat = one_hot_encoder(df_bureau_balance, nan_as_category)
    df_bureau, bureau_cat = one_hot_encoder(df_bureau, nan_as_category)

    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size', 'mean']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    # Status of Credit Bureau loan during the month
    bb_agg = df_bureau_balance.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    df_bureau = df_bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean', 'min'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean', 'max', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean', 'sum'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        'SK_ID_BUREAU': ['count'],
        'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],
        'ENDDATE_DIF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_FACT_DIFF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_ENDDATE_DIFF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_DEBT_RATIO': ['min', 'max', 'mean'],
        'DEBT_CREDIT_DIFF': ['min', 'max', 'mean'],
        'BUREAU_IS_DPD': ['mean', 'sum'],
        'BUREAU_IS_DPD_OVER120': ['mean', 'sum']
        }

    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    df_bureau_agg = df_bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    df_bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in df_bureau_agg.columns.tolist()])

    # Bureau: Active credits - using only numerical aggregations
    active = df_bureau[df_bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    df_bureau_agg = df_bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

    # Bureau: Closed credits - using only numerical aggregations
    closed = df_bureau[df_bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    df_bureau_agg = df_bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')

    print('"Bureau/Bureau Balance" final shape:', df_bureau_agg.shape)

"Bureau/Bureau Balance" final shape: (305811, 200)
Process bureau and bureau_balance - done in 18s


In [12]:
# Preprocess previous_applications.csv
with timer("Process previous_applications"):
    df_previous, cat_cols = one_hot_encoder(df_previous, nan_as_category=True)

    # Days 365.243 values -> nan
    df_previous['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    df_previous['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    df_previous['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    df_previous['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    df_previous['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

    # Add feature: value ask / value received percentage
    df_previous['APP_CREDIT_PERC'] = df_previous['AMT_APPLICATION'] / df_previous['AMT_CREDIT']

    # Feature engineering: ratios and difference
    df_previous['APPLICATION_CREDIT_DIFF'] = df_previous['AMT_APPLICATION'] - df_previous['AMT_CREDIT']
    df_previous['CREDIT_TO_ANNUITY_RATIO'] = df_previous['AMT_CREDIT'] / df_previous['AMT_ANNUITY']
    df_previous['DOWN_PAYMENT_TO_CREDIT'] = df_previous['AMT_DOWN_PAYMENT'] / df_previous['AMT_CREDIT']

    # Interest ratio on previous application (simplified)
    total_payment = df_previous['AMT_ANNUITY'] * df_previous['CNT_PAYMENT']
    df_previous['SIMPLE_INTERESTS'] = (total_payment / df_previous['AMT_CREDIT'] - 1) / df_previous['CNT_PAYMENT']

    # Days last due difference (scheduled x done)
    df_previous['DAYS_LAST_DUE_DIFF'] = df_previous['DAYS_LAST_DUE_1ST_VERSION'] - df_previous['DAYS_LAST_DUE']

    # from off
    df_previous['PREV_GOODS_DIFF'] = df_previous['AMT_APPLICATION'] - df_previous['AMT_GOODS_PRICE']
    df_previous['PREV_ANNUITY_APPL_RATIO'] = df_previous['AMT_ANNUITY'] / df_previous['AMT_APPLICATION']
    df_previous['PREV_GOODS_APPL_RATIO'] = df_previous['AMT_GOODS_PRICE'] / df_previous['AMT_APPLICATION']

    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean', 'sum'],
        'AMT_APPLICATION': ['min', 'max', 'mean', 'sum'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'sum'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'sum'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'SK_ID_PREV': ['nunique'],
        'DAYS_TERMINATION': ['max'],
        'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
        'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'sum'],
        'DOWN_PAYMENT_TO_CREDIT': ['mean'],
        'PREV_GOODS_DIFF': ['mean', 'max', 'sum'],
        'PREV_GOODS_APPL_RATIO': ['mean', 'max'],
        'DAYS_LAST_DUE_DIFF': ['mean', 'max', 'sum'],
        'SIMPLE_INTERESTS': ['mean', 'max']
    }

    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']

    df_previous_agg = df_previous.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    df_previous_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in df_previous_agg.columns.tolist()])

    # Previous Applications: Approved Applications - only numerical features
    approved = df_previous[df_previous['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    df_previous_agg = df_previous_agg.join(approved_agg, how='left', on='SK_ID_CURR')

    # Previous Applications: Refused Applications - only numerical features
    refused = df_previous[df_previous['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    df_previous_agg = df_previous_agg.join(refused_agg, how='left', on='SK_ID_CURR')

    print("Previous applications df shape:", df_previous_agg.shape)

Previous applications df shape: (338857, 321)
Process previous_applications - done in 28s


In [13]:
with timer("Process POS-CASH balance"):
    df_pos_cash, cat_cols = one_hot_encoder(df_pos_cash, nan_as_category=True)

    # Flag months with late payment
    df_pos_cash['LATE_PAYMENT'] = df_pos_cash['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    df_pos_cash['POS_IS_DPD'] = df_pos_cash['SK_DPD'].apply(lambda x: 1 if x > 0 else 0) # <-- same with ['LATE_PAYMENT']
    df_pos_cash['POS_IS_DPD_UNDER_120'] = df_pos_cash['SK_DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    df_pos_cash['POS_IS_DPD_OVER_120'] = df_pos_cash['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)

    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size', 'min'],
        'SK_DPD': ['max', 'mean', 'sum', 'var', 'min'],
        'SK_DPD_DEF': ['max', 'mean', 'sum'],
        'SK_ID_PREV': ['nunique'],
        'LATE_PAYMENT': ['mean'],
        'SK_ID_CURR': ['count'],
        'CNT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
        'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean', 'sum'],
        'POS_IS_DPD': ['mean', 'sum'],
        'POS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'POS_IS_DPD_OVER_120': ['mean', 'sum'],
    }

    for cat in cat_cols:
        aggregations[cat] = ['mean']

    pos_agg = df_pos_cash.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])

    # Count pos cash accounts
    pos_agg['POS_COUNT'] = df_pos_cash.groupby('SK_ID_CURR').size()


    sort_pos = df_pos_cash.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.groupby('SK_ID_PREV')
    df_pos = pd.DataFrame()
    df_pos['SK_ID_CURR'] = gp['SK_ID_CURR'].first()
    df_pos['MONTHS_BALANCE_MAX'] = gp['MONTHS_BALANCE'].max()

    # Percentage of previous loans completed and completed before initial term
    df_pos['POS_LOAN_COMPLETED_MEAN'] = gp['NAME_CONTRACT_STATUS_Completed'].mean()
    df_pos['POS_COMPLETED_BEFORE_MEAN'] = gp['CNT_INSTALMENT'].first() - gp['CNT_INSTALMENT'].last()
    df_pos['POS_COMPLETED_BEFORE_MEAN'] = df_pos.apply(lambda x: 1 if x['POS_COMPLETED_BEFORE_MEAN'] > 0 \
                                                                      and x['POS_LOAN_COMPLETED_MEAN'] > 0 else 0, axis=1)
    # Number of remaining installments (future installments) and percentage from total
    df_pos['POS_REMAINING_INSTALMENTS'] = gp['CNT_INSTALMENT_FUTURE'].last()
    df_pos['POS_REMAINING_INSTALMENTS_RATIO'] = gp['CNT_INSTALMENT_FUTURE'].last()/gp['CNT_INSTALMENT'].last()

    # Group by SK_ID_CURR and merge
    df_gp = df_pos.groupby('SK_ID_CURR').sum().reset_index()
    df_gp.drop(['MONTHS_BALANCE_MAX'], axis=1, inplace= True)
    pos_agg = pd.merge(pos_agg, df_gp, on= 'SK_ID_CURR', how= 'left')

    # Percentage of late payments for the 3 most recent applications
    pos = do_sum(df_pos_cash, ['SK_ID_PREV'], 'LATE_PAYMENT', 'LATE_PAYMENT_SUM')

    # Last month of each application
    last_month_df = pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()

    # Most recent applications (last 3)
    sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.iloc[last_month_df].groupby('SK_ID_CURR').tail(3)
    gp_mean = gp.groupby('SK_ID_CURR').mean().reset_index()
    pos_agg = pd.merge(pos_agg, gp_mean[['SK_ID_CURR', 'LATE_PAYMENT_SUM']], on='SK_ID_CURR', how='left')

    print('"Pos-Cash" balance final shape:', pos_agg.shape) 

"Pos-Cash" balance final shape: (337252, 46)
Process POS-CASH balance - done in 109s


In [ ]:
# Preprocess installments_payments.csv
# df_installments = pd.read_csv(path + 'installments_payments.csv')

with timer("Process installments payments"):
    df_installments, cat_cols = one_hot_encoder(df_installments, nan_as_category=True)

    # Group payments and get Payment difference
    df_installments = do_sum(df_installments, ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], 'AMT_PAYMENT', 'AMT_PAYMENT_GROUPED')
    df_installments['PAYMENT_DIFFERENCE'] = df_installments['AMT_INSTALMENT'] - df_installments['AMT_PAYMENT_GROUPED']
    df_installments['PAYMENT_RATIO'] = df_installments['AMT_INSTALMENT'] / df_installments['AMT_PAYMENT_GROUPED']
    df_installments['PAID_OVER_AMOUNT'] = df_installments['AMT_PAYMENT'] - df_installments['AMT_INSTALMENT']
    df_installments['PAID_OVER'] = (df_installments['PAID_OVER_AMOUNT'] > 0).astype(int)

    # Percentage and difference paid in each installment (amount paid and installment value)
    df_installments['PAYMENT_PERC'] = df_installments['AMT_PAYMENT'] / df_installments['AMT_INSTALMENT']
    df_installments['PAYMENT_DIFF'] = df_installments['AMT_INSTALMENT'] - df_installments['AMT_PAYMENT']

    # Days past due and days before due (no negative values)
    df_installments['DPD_diff'] = df_installments['DAYS_ENTRY_PAYMENT'] - df_installments['DAYS_INSTALMENT']
    df_installments['DBD_diff'] = df_installments['DAYS_INSTALMENT'] - df_installments['DAYS_ENTRY_PAYMENT']
    df_installments['DPD'] = df_installments['DPD_diff'].apply(lambda x: x if x > 0 else 0)
    df_installments['DBD'] = df_installments['DBD_diff'].apply(lambda x: x if x > 0 else 0)

    # Flag late payment
    df_installments['LATE_PAYMENT'] = df_installments['DBD'].apply(lambda x: 1 if x > 0 else 0)
    df_installments['INSTALMENT_PAYMENT_RATIO'] = df_installments['AMT_PAYMENT'] / df_installments['AMT_INSTALMENT']
    df_installments['LATE_PAYMENT_RATIO'] = df_installments.apply(lambda x: x['INSTALMENT_PAYMENT_RATIO'] if x['LATE_PAYMENT'] == 1 else 0, axis=1)

    # Flag late payments that have a significant amount
    df_installments['SIGNIFICANT_LATE_PAYMENT'] = df_installments['LATE_PAYMENT_RATIO'].apply(lambda x: 1 if x > 0.05 else 0)
    
    # Flag k threshold late payments
    df_installments['DPD_7'] = df_installments['DPD'].apply(lambda x: 1 if x >= 7 else 0)
    df_installments['DPD_15'] = df_installments['DPD'].apply(lambda x: 1 if x >= 15 else 0)

    df_installments['INS_IS_DPD_UNDER_120'] = df_installments['DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    df_installments['INS_IS_DPD_OVER_120'] = df_installments['DPD'].apply(lambda x: 1 if (x >= 120) else 0)

    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum', 'var'],
        'DBD': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum', 'min'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum', 'min'],
        'SK_ID_PREV': ['size', 'nunique'],
        'PAYMENT_DIFFERENCE': ['mean'],
        'PAYMENT_RATIO': ['mean', 'max'],
        'LATE_PAYMENT': ['mean', 'sum'],
        'SIGNIFICANT_LATE_PAYMENT': ['mean', 'sum'],
        'LATE_PAYMENT_RATIO': ['mean'],
        'DPD_7': ['mean'],
        'DPD_15': ['mean'],
        'PAID_OVER': ['mean'],
        'DPD_diff':['mean', 'min', 'max'],
        'DBD_diff':['mean', 'min', 'max'],
        'DAYS_INSTALMENT': ['mean', 'max', 'sum'],
        'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'INS_IS_DPD_OVER_120': ['mean', 'sum']
    }

    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = df_installments.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = df_installments.groupby('SK_ID_CURR').size()

    # from oof (DAYS_ENTRY_PAYMENT)
    cond_day = df_installments['DAYS_ENTRY_PAYMENT'] >= -365
    ins_d365_grp = df_installments[cond_day].groupby('SK_ID_CURR')
    ins_d365_agg_dict = {
        'SK_ID_CURR': ['count'],
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DAYS_ENTRY_PAYMENT': ['mean', 'max', 'sum'],
        'DAYS_INSTALMENT': ['mean', 'max', 'sum'],
        'AMT_INSTALMENT': ['mean', 'max', 'sum'],
        'AMT_PAYMENT': ['mean', 'max', 'sum'],
        'PAYMENT_DIFF': ['mean', 'min', 'max', 'sum'],
        'PAYMENT_PERC': ['mean', 'max'],
        'DPD_diff': ['mean', 'min', 'max'],
        'DPD': ['mean', 'sum'],
        'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'INS_IS_DPD_OVER_120': ['mean', 'sum']}

    ins_d365_agg = ins_d365_grp.agg(ins_d365_agg_dict)
    ins_d365_agg.columns = ['INS_D365' + ('_').join(column).upper() for column in ins_d365_agg.columns.ravel()]

    ins_agg = ins_agg.merge(ins_d365_agg, on='SK_ID_CURR', how='left')

    print('"Installments Payments" final shape:', ins_agg.shape)

In [ ]:
# Preprocess credit_card_balance.csv
with timer("Process credit card balance"):
    df_credit_card, cat_cols = one_hot_encoder(df_credit_card, nan_as_category=True)

    # Amount used from limit
    df_credit_card['LIMIT_USE'] = df_credit_card['AMT_BALANCE'] / df_credit_card['AMT_CREDIT_LIMIT_ACTUAL']
    # Current payment / Min payment
    df_credit_card['PAYMENT_DIV_MIN'] = df_credit_card['AMT_PAYMENT_CURRENT'] / df_credit_card['AMT_INST_MIN_REGULARITY']
    # Late payment <-- 'CARD_IS_DPD'
    df_credit_card['LATE_PAYMENT'] = df_credit_card['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    # How much drawing of limit
    df_credit_card['DRAWING_LIMIT_RATIO'] = df_credit_card['AMT_DRAWINGS_ATM_CURRENT'] / df_credit_card['AMT_CREDIT_LIMIT_ACTUAL']

    df_credit_card['CARD_IS_DPD_UNDER_120'] = df_credit_card['SK_DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    df_credit_card['CARD_IS_DPD_OVER_120'] = df_credit_card['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)

    # General aggregations
    cc_agg = df_credit_card.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])

    # Count credit card lines
    cc_agg['CC_COUNT'] = df_credit_card.groupby('SK_ID_CURR').size()

    # Last month balance of each credit card application
    last_ids = df_credit_card.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()
    last_months_df = df_credit_card[df_credit_card.index.isin(last_ids)]
    cc_agg = group_and_merge(last_months_df, cc_agg, 'CC_LAST_', {'AMT_BALANCE': ['mean', 'max']})

    CREDIT_CARD_TIME_AGG = {
        'AMT_BALANCE': ['mean', 'max'],
        'LIMIT_USE': ['max', 'mean'],
        'AMT_CREDIT_LIMIT_ACTUAL':['max'],
        'AMT_DRAWINGS_ATM_CURRENT': ['max', 'sum'],
        'AMT_DRAWINGS_CURRENT': ['max', 'sum'],
        'AMT_DRAWINGS_POS_CURRENT': ['max', 'sum'],
        'AMT_INST_MIN_REGULARITY': ['max', 'mean'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['max','sum'],
        'AMT_TOTAL_RECEIVABLE': ['max', 'mean'],
        'CNT_DRAWINGS_ATM_CURRENT': ['max','sum', 'mean'],
        'CNT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],
        'CNT_DRAWINGS_POS_CURRENT': ['mean'],
        'SK_DPD': ['mean', 'max', 'sum'],
        'LIMIT_USE': ['min', 'max'],
        'DRAWING_LIMIT_RATIO': ['min', 'max'],
        'LATE_PAYMENT': ['mean', 'sum'],
        'CARD_IS_DPD_UNDER_120': ['mean', 'sum'],
        'CARD_IS_DPD_OVER_120': ['mean', 'sum']
    }

    for months in [12, 24, 48]:
        cc_prev_id = df_credit_card[df_credit_card['MONTHS_BALANCE'] >= -months]['SK_ID_PREV'].unique()
        cc_recent = df_credit_card[df_credit_card['SK_ID_PREV'].isin(cc_prev_id)]
        prefix = 'INS_{}M_'.format(months)
        cc_agg = group_and_merge(cc_recent, cc_agg, prefix, CREDIT_CARD_TIME_AGG)

    print('"Credit Card Balance" final shape:', cc_agg.shape)

In [ ]:
print("Applications df shape:", df_applications.shape)
print("Bureau df shape:", df_bureau_agg.shape)
df = df_applications.merge(df_bureau_agg, how='left', on='SK_ID_CURR')
print("df shape after merge of Applications and Bureau :", df.shape)

In [ ]:
print("Previous applications df shape:", df_previous_agg.shape)
df = df.merge(df_previous_agg, how='left', on='SK_ID_CURR')
print("df shape after merge with Previous applications :", df.shape)

In [ ]:
print("Pos-cash balance df shape:", pos_agg.shape)
df = df.merge(pos_agg, how='left', on='SK_ID_CURR')
print("df shape after merge with Pos-cash balance :", df.shape)

In [ ]:
print("Installments payments df shape:", ins_agg.shape)
df = df.merge(ins_agg, how='left', on='SK_ID_CURR')
print("df shape after merge with Installments payments :", df.shape)

In [ ]:
print("Credit card balance df shape:", cc_agg.shape)
df = df.merge(cc_agg, how='left', on='SK_ID_CURR')
print("df shape after merge with Credit card :", df.shape)

In [ ]:
df.head()

In [ ]:
df.to_csv('df_complet.csv', index=False)

In [ ]:
df.shape

In [ ]:
# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
stratified = False
debug = False
num_folds = 10
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
with timer("Run LightGBM with kfold"):
    # Divide in training/validation and test data
    X = df.loc[:, df.columns != 'TARGET']
    y = df['TARGET']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    train_df = X_train.join(y_train)
    test_df = X_test.join(y_test)
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)

    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1)

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric='auc', verbose=200, early_stopping_rounds=200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    display_importances(feature_importance_df)

In [ ]:
cols = feature_importance_df[["feature", "importance"]].groupby(
    "feature").mean().sort_values(by="importance", ascending=False)[:800].index
best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]
best_features.to_csv('best_features.csv', index=False)

In [ ]:
best_features = pd.read_csv('best_features.csv')
print(best_features.shape)

In [ ]:
colstosave = best_features['feature'].unique()
colonnes = []
for col in df.columns:
    if col in colstosave:
        colonnes.append(col)
colonnes.append('TARGET')

In [ ]:
df_final = df[colonnes]
print(df_final.shape)

In [ ]:
df_final.to_csv('df_final.csv', index=False)